In [59]:
import os

from astropy.table import Table
from astropy.io import ascii
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u

import numpy as np

import pandas as pd

import sfdmap

In [61]:
# from dustmaps.config import config
# config['data_dir'] = '/home/mj1e16/DUST_DIR/'

# import dustmaps.sfd
# dustmaps.sfd.fetch()

In [66]:
schalf_dust_map = sfdmap.SFDMap('/home/mj1e16/DUST_DIR/sfd')

In [2]:
databaseDir = '/home/mj1e16/outTablesImproved/extendedMagRange/'
imdir = r'/media/mj1e16/PP AV-TV/keplerCal/'

In [3]:
dirList = os.listdir(databaseDir)
databases = [x for x in dirList if '.csv' in x]
databases = [databaseDir+x for x in databases]
databases.sort()

In [4]:
imageList = os.listdir(imdir)
imageList = [x for x in imageList if '.fits' in x]
imageList = [imdir+x for x in imageList]

In [5]:
def extractQuality(table):
    tab = ascii.read(table,format='csv')
    df = tab.to_pandas()
    dfselected = df[(df['detectMinarea']==7.0)&(df['detectThresh']==7.0)&(df['filterName']=='default.conv')]
    comp = df['completenessScore'].values[0]
    acc = df['accuracyScore'].values[0]
    return comp, acc

In [6]:
compList = {}
accList = {}

y = 0

for x in range(len(databases)):
    comp,acc = extractQuality(databases[x])
    compList[y] = comp
    accList[y] = acc
    #print(comp)
    if y >= 0:
        y +=1
    else:
        y += -1
    if y > 7:
        y = -1

In [47]:
compList

{-7: 0.6905241935483871,
 -6: 0.11995967741935487,
 -5: 0.11895161290322576,
 -4: 0.12096774193548387,
 -3: 0.3578629032258065,
 -2: 0.20967741935483875,
 -1: 0.1401209677419355,
 0: 0.7439516129032258,
 1: 0.9415322580645161,
 2: 0.9899193548387096,
 3: 1.0,
 4: 1.0,
 5: 1.0,
 6: 1.0,
 7: 1.0}

In [7]:
def makeImageCoordinates(image,boxlength=100):
    hdu_list = fits.open(image)
    for ccd in range(1,len(hdu_list)):
        image_shape = hdu_list[ccd].data.shape
        w = WCS(hdu_list[ccd])
        xiterrations = (image_shape[0]-(image_shape[0]%boxlength))/boxlength
        yiterrations = (image_shape[1]-(image_shape[1]%boxlength))/boxlength
        xVal = boxlength/2
        yVal = boxlength/2
        wx, wy = w.wcs_pix2world(xVal, yVal, 1)
        locs = [(wx,wy)]
        for x in range(xiterrations):
            yVal = boxlength/2
            xVal += boxlength
            for y in range(yiterrations):
                yVal += boxlength 
                wx, wy = w.wcs_pix2world(xVal, yVal, 1)
                locs.append((wx,wy))
        
    return locs

In [8]:
# coords_list = []
# for x in imageList:
#     coords_list.append(makeImageCoordinates(x))

In [9]:
image = r'/media/mj1e16/PP AV-TV/keplerCal/kplr2009114174833_ffi-cal.fits'

In [10]:
coords = makeImageCoordinates(image)

In [11]:
def findWeight(l,b,r):
    rogrim,rtgrim,rdgrim,rzgrim,rmgrim,gamma,Rc,q = 1.,1.9,3.5,0.41,6.5,1.8,2.8,0.6
    milkyWayRadius = 15.
    scaleHeight = 3.5
    bSphere = 7.669
    ebv=[]
    Rd = 3 

    radius = r * np.cos(np.deg2rad(b))
    thickness = r * np.sin(np.deg2rad(b))
                   
    xlen = (((radius*np.sin(np.deg2rad(l)))))
    ylen = ((radius*np.cos(np.deg2rad(l)))-8.364)
    dist = ((xlen**2)+(ylen**2))**0.5
    rad = ((dist**2)+(thickness**2))**0.5
    
    Bulge = (((((dist**2)+((thickness**2)/(q**2)))**0.5)/rogrim)**(-gamma))*np.exp(-(((dist**2)+((thickness**2)/(q**2)))/rtgrim**2))
    Disc = (np.exp((-rmgrim/rdgrim)-(dist/rdgrim)-(abs(thickness)/rzgrim)))
    Sphere = (np.exp(-bSphere*((rad/Rc)**0.25)))/((rad/Rc)**(7./8.))
    
    results = {}
    results[0] = [xlen,ylen,thickness]
    results[5] = [Bulge,Disc,Sphere]
    
    return results

In [12]:
def threeDintegrate(l,b,r,dr,db,dl):
    l = 2*np.pi*(l/360.)
    dl = 2*np.pi*(dl/360.)
    b = 2*np.pi*(b/360.)
    db = 2*np.pi*(db/360.)
    
    #phi = (2*np.pi*(90/360)) - b
    #print(np.sin(l))
    volume = abs((r**2)*np.sin(l)*dl*dr*db)
    return volume

In [29]:
def findRegionProbs(coords,radialDistance):
    weightPerAngle = []
    volumePerAngle = []

    # buldgeCumulative = 0
    # discCumulative = 0
    # sphereCumulative = 0

    volumeCumulative = 0

    for ra_dec in coords:
        c = SkyCoord(ra_dec[0], ra_dec[1], frame='icrs', unit='deg')
        l = c.galactic.l.deg
        b = c.galactic.b.deg
        weightPerDistance = []
        volumePerDistance = []
        for dist in radialDistance:
            results = findWeight(l,b,dist)
            weightPerSect = results[5]
            weightPerDistance.append(weightPerSect)

    #         buldgeCumulative += weightPerSect[0]
    #         discCumulative += weightPerSect[1]
    #         sphereCumulative += weightPerSect[2]

            volumePerSect = threeDintegrate(l,b,dist,results[0][0],results[0][1],results[0][2])
            volumeCumulative += volumePerSect
            volumePerDistance.append(volumePerSect)

        weightPerAngle.append(weightPerDistance)
        volumePerAngle.append(volumePerDistance)
        
    return volumePerAngle,weightPerAngle

In [63]:
def findRegionProbsandCompleteness(coords,radialDistance,absMag,lumFunc,completeness,R=3.1):
    weightPerAngle = []
    volumePerAngle = []

    # buldgeCumulative = 0
    # discCumulative = 0
    # sphereCumulative = 0
    totStars = 0
    totStarsFound = 0
    volumeCumulative = 0
    badkeys = []
    completenessKeys = completeness.keys()
    for ra_dec in coords:
        c = SkyCoord(ra_dec[0], ra_dec[1], frame='icrs', unit='deg')
        l = c.galactic.l.deg
        b = c.galactic.b.deg
        ebv = schalf_dust_map.ebv(l,b, frame='galactic', unit='degree', interpolate=False)
        A = R * ebv
        weightPerDistance = []
        volumePerDistance = []
        for dist in radialDistance:
            results = findWeight(l,b,dist)
            weightPerSect = results[5]
            weightPerDistance.append(weightPerSect)
                    
            volumePerSect = threeDintegrate(l,b,dist,results[0][0],results[0][1],results[0][2])
            volumeCumulative += volumePerSect
            volumePerDistance.append(volumePerSect)
            
            for magRange in range(len(absMag)):
            
                apparentMag = absMag[magRange] + (5*(np.log10(dist)-1)) + A
                starNo = np.exp(lumFunc[magRange]) * volumePerSect 
                compKey = float(int(apparentMag-18.6))
                if compKey in completenessKeys:
                    starsFound = starNo * (1-completeness[compKey])
                else:
                    starsFound = 0
                    badkeys.append(compKey)
                totStars += starNo
                totStarsFound += starsFound
                
        weightPerAngle.append(weightPerDistance)
        volumePerAngle.append(volumePerDistance)
    
    compForwholeReg = float(totStarsFound)/float(totStars)
    
    return [volumePerAngle,weightPerAngle,totStars,compForwholeReg,badkeys]

In [64]:
absoluteMag = np.linspace(-7,18,26)
singleStars = [-7.98,-7.60,-7.27,-6.72,-6.05,-5.43,-4.8,-4.18,-3.6,-3.16,-2.89,-2.63,-2.49,
               -2.44,-2.52,-2.41,-2.32,-2.14,-1.99,-1.82,-1.9,-2.0,-2.0,-2.1,-2.1,-2.2]

In [67]:
results = findRegionProbsandCompleteness(coords,radialDistance,absoluteMag,singleStars,compList)

In [71]:
print(max(results[-1]))

9.0


In [30]:
vpa, wpa = findRegionProbs(coords,radialDistance)

In [81]:
c = SkyCoord(coords[0][0], coords[0][1], frame='icrs', unit='deg')

In [13]:
radialDistance = np.linspace(0.1,130.,60)#noSteps)#15)

In [14]:
weightPerAngle = []
volumePerAngle = []

# buldgeCumulative = 0
# discCumulative = 0
# sphereCumulative = 0

volumeCumulative = 0

for ra_dec in coords:
    c = SkyCoord(ra_dec[0], ra_dec[1], frame='icrs', unit='deg')
    l = c.galactic.l.deg
    b = c.galactic.b.deg
    weightPerDistance = []
    volumePerDistance = []
    for dist in radialDistance:
        results = findWeight(l,b,dist)
        weightPerSect = results[5]
        weightPerDistance.append(weightPerSect)
        
#         buldgeCumulative += weightPerSect[0]
#         discCumulative += weightPerSect[1]
#         sphereCumulative += weightPerSect[2]
        
        volumePerSect = threeDintegrate(l,b,dist,results[0][0],results[0][1],results[0][2])
        volumeCumulative += volumePerSect
        volumePerDistance.append(volumePerSect)
        
    weightPerAngle.append(weightPerDistance)
    volumePerAngle.append(volumePerDistance)

In [18]:
weightPerAngle[0][0]

[9.092606473209177e-11, 0.01399050135190474, 1.6161165872166317e-05]

In [15]:
noSteps = 60
#b = [0]
bSteps = 100
lSteps = 100
rSteps = 260

b = np.linspace(-90.,90,bSteps)
l = np.linspace(0., 359., lSteps)
radialDistance = np.linspace(0.1,130.,rSteps)#noSteps)#15)



dr = (radialDistance[-1]-radialDistance[0])/rSteps
dl = (l[-1]-(l[0]))/lSteps
db =(b[-1]-(b[0]))/bSteps

l,b = [0],[0]

volumeSegment = []
#rM = 5
probabilities = []

volVal = 0

#magrange = np.linspace(13,22,20)
resultList = []
for x,angle in enumerate(l):
    resultPerB = []
    for lat,lattitude in enumerate(b):
        resultPerD = []
        for y,item in enumerate(radialDistance):
            radius = item * np.cos(np.deg2rad(lattitude))
            xlen = (((radius*np.sin(np.deg2rad(angle)))))
            ylen = ((radius*np.cos(np.deg2rad(angle)))-8.364)
            Dist = ((xlen**2)+(ylen**2))**0.5
            width = Dist
            thickness = item * np.sin(np.deg2rad(lattitude))
            distancefromcentre = ((thickness**2)+(Dist**2))**0.5
            if abs(width) < 100 and abs(thickness) < 100:# and distancefromcentre > 1.:
                result = findWeight(angle,lattitude,item)
                probabilities.append(result[5])
                volSeg = threeDintegrate(angle,lattitude,item,dr,dl,db)
                volumeSegment.append(volSeg)
                volVal += volSeg
            else:
                print(width,thickness)
                
bulgecumalative = 0
disccumalative = 0
spherecumalative = 0

bulgeProb = []
discProb = []
sphereProb = []



for x in range(len(probabilities)):
    bulgecumalative += probabilities[x][0]#*volumeSegment[x]
    disccumalative += probabilities[x][1]#*volumeSegment[x]
    spherecumalative += probabilities[x][2]#*volumeSegment[x]
    
    bulgeProb.append(probabilities[x][0])#*volumeSegment[x])
    discProb.append(probabilities[x][1])#*volumeSegment[x])
    sphereProb.append(probabilities[x][2])#*volumeSegment[x])

totcumulative = bulgecumalative + disccumalative + spherecumalative
# Relative Mass Ratios are decided Below
discnormalisation = (2.*totcumulative)/(3.8*disccumalative)
bulgenormalisation = (1.*totcumulative)/(3.8*bulgecumalative)
spherenormalisation = (0.8*totcumulative)/(3.8*spherecumalative)

discProb = [x*discnormalisation for x in discProb]
bulgeProb = [x*bulgenormalisation for x in bulgeProb]
sphereProb = [x*spherenormalisation for x in sphereProb]

totProb = [a+b+c for a,b,c in zip(discProb,bulgeProb,sphereProb)]
print(totProb)
print(sum(totProb))
totSum = sum(totProb)
totProb = [x/totSum for x in totProb]
print(sum(totProb))

(100.06959073359074, 0.0)
(100.57113513513514, 0.0)
(101.07267953667954, 0.0)
(101.57422393822394, 0.0)
(102.07576833976835, 0.0)
(102.57731274131274, 0.0)
(103.07885714285715, 0.0)
(103.58040154440155, 0.0)
(104.08194594594595, 0.0)
(104.58349034749035, 0.0)
(105.08503474903475, 0.0)
(105.58657915057915, 0.0)
(106.08812355212356, 0.0)
(106.58966795366796, 0.0)
(107.09121235521236, 0.0)
(107.59275675675676, 0.0)
(108.09430115830116, 0.0)
(108.59584555984557, 0.0)
(109.09738996138996, 0.0)
(109.59893436293437, 0.0)
(110.10047876447877, 0.0)
(110.60202316602317, 0.0)
(111.10356756756757, 0.0)
(111.60511196911197, 0.0)
(112.10665637065638, 0.0)
(112.60820077220077, 0.0)
(113.10974517374518, 0.0)
(113.61128957528958, 0.0)
(114.11283397683398, 0.0)
(114.61437837837839, 0.0)
(115.11592277992278, 0.0)
(115.61746718146719, 0.0)
(116.11901158301158, 0.0)
(116.62055598455599, 0.0)
(117.12210038610039, 0.0)
(117.62364478764479, 0.0)
(118.1251891891892, 0.0)
(118.6267335907336, 0.0)
(119.128277992

In [16]:
print(disccumalative*discnormalisation)
print(bulgecumalative*bulgenormalisation)
print(spherecumalative*spherenormalisation)

15.998514687244192
7.999257343622095
6.399405874897678


In [17]:
print(disccumalative)
print(bulgecumalative)
print(spherecumalative)
print(discnormalisation)
print(bulgenormalisation)
print(spherenormalisation)

2.081507409080398
28.024492593782917
0.291177902900651
7.686023416228085
0.2854380794532811
21.977649440936922


In [96]:
totProb


[0.11353544038026946,
 0.13105797351381396,
 0.15129367673672092,
 0.17466711546417596,
 0.20167209488191254,
 0.23288536092672,
 0.2689862940479196,
 0.31079066244886083,
 0.35932079982075255,
 0.4159694076909244,
 0.4828915310159007,
 0.563938312425582,
 0.6669852786202337,
 0.8108000455187453,
 1.052927141911182,
 1.6791632621450292,
 7.798185342951616,
 6.736884211853615,
 1.6274004705913452,
 1.0376650444492936,
 0.8027919042559669,
 0.6615895375421325,
 0.5598305949457794,
 0.4795577272057535,
 0.41317176484849005,
 0.35693387796135756,
 0.30873835181815174,
 0.267215498455993,
 0.2313549709775169,
 0.20034836443553142,
 0.17352157726929884,
 0.15030202806318432,
 0.1301993510093411,
 0.1127918798708645,
 0.09771618262795521,
 0.08465855850522133,
 0.07334796420618926,
 0.06355005082535463,
 0.05506209537157162,
 0.047708669064655866,
 0.04133792146313657,
 0.03581838457941722,
 0.03103621915594409,
 0.026892838785622084,
 0.023302858049140878,
 0.020192319210920674,
 0.017497158

In [92]:
weightPerAngle[0][0]

{0: [0.09786550400003544, -8.346759628422824, 0.011185379506297537],
 5: [9.092606473209177e-11, 0.01399050135190474, 1.6161165872166317e-05]}

In [20]:
keplerBtot = 0
keplerDtot = 0
keplerStot = 0

for ang in range(len(weightPerAngle)):
    for dist in range(len(weightPerAngle[ang])):
        keplerBtot += weightPerAngle[ang][dist][0]
        keplerDtot += weightPerAngle[ang][dist][1]
        keplerStot += weightPerAngle[ang][dist][2]
        
normKeplerBtot = keplerBtot*bulgenormalisation
normKeplerDtot = keplerDTot*discnormalisation
normKeplerStot = keplerStot*spherenormalisation

In [23]:
normKeplerStot

0.17667814884443328

In [24]:
normKeplerStot/31.

0.005699295124013977

In [27]:
print(sum(sphereProb))

6.39940587489768


In [ ]:
# need to test half total galaxy with different spacing

In [56]:
def findRegionProbsandCompleteness(coords,radialDistance,absMag,lumFunc,completeness,R=3.1):
    weightPerAngle = []
    volumePerAngle = []

    # buldgeCumulative = 0
    # discCumulative = 0
    # sphereCumulative = 0
    totStars = 0
    totStarsFound = 0
    volumeCumulative = 0
    badkeys = []
    completenessKeys = completeness.keys()
    for ra_dec in coords:
        c = SkyCoord(ra_dec[0], ra_dec[1], frame='icrs', unit='deg')
        l = c.galactic.l.deg
        b = c.galactic.b.deg
        ebv = schalf_dust_map.ebv(l,b, frame='galactic', unit='degree', interpolate=False)
        A = R * ebv
        weightPerDistance = []
        volumePerDistance = []
        for dist in radialDistance:
            results = findWeight(l,b,dist)
            weightPerSect = results[5]
            weightPerDistance.append(weightPerSect)
                    
            volumePerSect = threeDintegrate(l,b,dist,results[0][0],results[0][1],results[0][2])
            volumeCumulative += volumePerSect
            volumePerDistance.append(volumePerSect)
            
            for magRange in range(len(absMag)):
            
                apparentMag = absMag[magRange] + (5*(np.log10(dist)-1)) + A
                starNo = np.exp(lumFunc[magRange]) * volumePerSect 
                compKey = float(int(apparentMag-18.6))
                if compKey in completenessKeys:
                    starsFound = starNo * (1-completeness[compKey])
                else:
                    starsFound = 0
                    badkeys.append(compKey)
                totStars += starNo
                totStarsFound += starsFound
                
        weightPerAngle.append(weightPerDistance)
        volumePerAngle.append(volumePerDistance)
    
    compForwholeReg = float(totStarsFound)/float(totStars)
    
    return volumePerAngle,weightPerAngle,totStars,compForwholeReg,badkeys

In [ ]:
ebv = schalf_dust_map.ebv(l,b, frame='galactic', unit='degree', interpolate=False)
R = 3.1
reddening = ((ebv)*R)

In [42]:
absoluteMag = np.linspace(-7,18,26)

In [43]:
absoluteMag

array([-7., -6., -5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,
        6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18.])

In [46]:
singleStars = [-7.98,-7.60,-7.27,-6.72,-6.05,-5.43,-4.8,-4.18,-3.6,-3.16,-2.89,-2.63,-2.49,
               -2.44,-2.52,-2.41,-2.32,-2.14,-1.99,-1.82,-1.9,-2.0,-2.0,-2.1,-2.1,-2.2]

print(len(singleStars))

26


In [50]:
float(int(11.7))

11.0

In [53]:
a = {}
a[1] = 0

In [55]:
print(a.keys())

[1]
